<h1>Gephi to Tableau</h1>

In [1]:
#add file name
fileName = "LM.gexf"

In [2]:
#@title <b><font color=#FFB81C>Step #2:</font> <font color=#154734>Process</font></b> { display-mode: "form" }
#@markdown <p>Run the code block to the left</p>

import pandas as pd
import xml.etree.ElementTree as ET
import ipywidgets as widgets

node_dict = {}

#opening file:
with open (fileName, 'r+', encoding = "utf8") as inFile:
    fileStr = inFile.read()
    inFile.close()

#replacing viz:
fileStr = fileStr.replace("viz:", "")

#identifying nodes and edges and putting into a string
nIdentifier = "</nodes>"
eIdentifier = "</edges>"
iBreak = fileStr.find(nIdentifier) + len(nIdentifier)

nodeStr = fileStr[fileStr.find("<nodes>") :  iBreak]
edgeStr = fileStr[iBreak: fileStr.find(eIdentifier) + len(eIdentifier)]


#EDGES
#extracting data of edges
eRoot = ET.fromstring(edgeStr)
edges = eRoot.findall("edge")
source_target = [(edge.attrib["id"], edge.attrib["source"], edge.attrib["target"], edge.get("weight")) for edge in edges]
dfEdges = pd.DataFrame(source_target, columns=['idEdge', 'source', 'target', 'weight'])
dfEdges = dfEdges.assign(path = 1)

#df 
df = dfEdges[['idEdge']].copy(deep=True)
df = df.join( dfEdges[['target']].copy(deep=True) )
df = df.assign(path = 2)
df = df.rename(columns={'target':'source'})

dfEdges.drop('target', axis=1, inplace=True)
dfEdges = pd.concat([dfEdges, df])

dfEdges = dfEdges.astype(float) #typecasting to floats


#nodes
tree = ET.ElementTree(ET.fromstring(nodeStr))    
root = tree.getroot()

# or directly from a string as
# root = ET.fromstring(string_name)

# iterating through the children of the root
id_label = ""
attvalue = {}

gotAtt = False #NEEDED

valueError = False #NEEDED
headersFound = False #logic of: if one of the other is found, turns true

node_elements = []
att_columns = []  # gather how many/what the different attvalue columns will be
# which will be determined before setting the attvalue dict
att_values = []  # gather the data for the column above


element_bank = {"size": 'value', "position": ["x", "y"], "color": ["r", "g", "b"]}
column_headers = []

# function that finds the value of a single attribute by splitting it amongst a colon
def value_attributeFinder(string_element, commaIndex, bool_afterComma):
    iStart = 0
    iEnd = -1

    if bool_afterComma == True:
        iStart = commaIndex
    else:
        iEnd = commaIndex

    iColon = string_element.find(':', iStart, iEnd)
    iFirstQ = string_element.find("'", iColon, iEnd)
    iSecondQ = string_element.find("'", iFirstQ + 1, iEnd)

    atty = string_element[iFirstQ + 1:iSecondQ]

    return atty

#identifying the unique elements to a node

for child in root.find('.//node'):
    if child.tag != "attvalues":
        node_elements.append(child.tag)
        if child.tag in element_bank:
            if str(type(element_bank[child.tag])) == "<class 'str'>":
                # extracts the exact column headers once
                if child.tag in column_headers:
                    headersFound = True

                if headersFound == False:
                    column_headers.append(child.tag)
                    
                specificCase = element_bank[child.tag]
                if str(child.attrib).find(specificCase) == -1:
                    valueError = True
                    
            # case for list (when the list has multiple values e.g. position)
            elif str(type(element_bank[child.tag])) == "<class 'list'>":
                for i in element_bank[child.tag]:
                    if i in column_headers:
                        headersFound = True

                    if headersFound == False:
                        column_headers.append(i)

            else:
                print("Error - node could not be found in input file")
                
                
    #finds whether the file has attvalues or not
    #logically, the statement below can be an elif, and then finally, else being undetermined (unforseen) elements 
    if gotAtt == False:
        gotAtt = root.find('.//attvalues')
        #print("PRINTING GOTATT BELOW:")
        if str(gotAtt) != "None":
            for child in root.find('.//attvalues'):
                attElement = str(child.attrib)
                iComma = attElement.find(',')
                # extracting substring surrounded by quotes up until the comma (FOR)
                att_columns.append(value_attributeFinder(attElement, iComma, False))
            

stringsElements = []
for i in node_elements:
    stringsElements.append([])
    
stringsAttvalues = []
for i in att_columns:
    stringsAttvalues.append([])


allocatedDicts = False
seenAttribs = []

#creating a nested list for the column values for each headers that were identified
column_values = []
for headers in column_headers:
    column_values.append([])
#instead of using a dict, the nested lists' index will correspond to the headers 

for child in root.iter():
    inAtt = False
    if str(child.tag) == "node":
        #below gets the two columns for id and label
        id_label = id_label + str(child.attrib) + ", "
    
    #section below that determines and extracts the elements 
    if child.tag in node_elements:
        #allocating variable space for the "first time"
        valuestring = str(child.attrib)
        
        if allocatedDicts != True:
            iFirstA = valuestring.find("'")
            iSecondA = valuestring.find("'", iFirstA + 1)
            currParam = valuestring[iFirstA + 1:iSecondA]
    
            if currParam in seenAttribs:
                allocatedDicts = True
            else:
                seenAttribs.append(currParam)
            
        for idx, elements in enumerate(node_elements):
            if elements in child.tag:
                stringsElements[idx].append(child.attrib)
    if child.tag == "attvalue":
        attElement = str(child.attrib)
        iComma = attElement.find(',')
        currCol = value_attributeFinder(attElement, iComma, False)
        #print("currCol: " + str(currCol))
        iCol = att_columns.index(currCol)
        stringsAttvalues[iCol].append(value_attributeFinder(attElement, iComma, True))
             
if valueError == True:
    print("note: the values in the xml file may have been parsed incorrectly")
    

def stringToDict(elementStr, endComma):
    if endComma == True:
        elementStr = elementStr[:-2]
    return eval(elementStr)

dString = id_label[:-2] #gets rid of the comma at the end
nodeDict = eval(dString)

#creating df of different child/attributes
dfNode = pd.DataFrame(nodeDict)

for idx, element in enumerate(stringsElements):
    #splicing
    tmp = str(element)
    stringsElements[idx] = tmp[1:-1]
    
    #joining dataframes
    dfTmp = pd.DataFrame( stringToDict(str(element), False) )
    dfNode = dfNode.join(dfTmp)


dfAtt = pd.DataFrame()
for i, col in enumerate(att_columns):
    dfAtt.insert(loc=i,column=col, value=stringsAttvalues[i])
    
dfNode = dfNode.join(dfAtt)

#cleaning the dfNode dataframe:
dfNode = dfNode.rename(columns={'id':'idNode'})
numericList = ['idNode', 'x', 'y', 'value', 'r', 'g', 'b', 'latitude', 'longitude']
for col in dfNode.columns:
    if(col in numericList):
        dfNode[col] = dfNode[col].astype(float)
    #as inside forloop, n-1 steps not needed if 'a' exists,
    #n steps not needed if 'a' DNE
    if(col == 'a'):
        dfNode.drop('a', axis=1, inplace=True)
        
#writing to an Excel file
outFile = fileName[0:fileName.find('.')]
with pd.ExcelWriter("TableauReady" + outFile + ".xlsx") as writer:
    dfNode.to_excel(writer, sheet_name='Nodes', index=False)
    dfEdges.to_excel(writer, sheet_name='Edges', index=False)

print("code reached the end without any warnings or unexpected errors")

def clicked(arg):
      files.download(writer)

#download widget 
layout = widgets.Layout(width='auto', height='40px') #set width and height
button_download = widgets.Button(description = 'Click to download Tableau compatible .xlsx file', layout = layout, button_style='success')   
button_download.on_click(clicked)
display(button_download)

code reached the end without any warnings or unexpected errors


Button(button_style='success', description='Click to download Tableau compatible .xlsx file', layout=Layout(he…